4.1: VAE

In [ ]:
import torch
import torchvision.transforms as transforms
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if not torch.cuda.is_available():
    print("Warning CUDA not Found. Using CPU")

# Hyper-parameters
num_epochs = 8
learning_rate = 1e-3

In [ ]:
class MRIDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith(".png")]
        self.transform = transform

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        image = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            image = self.transform(image)
        return image, 0 # Return image and a dummy label
# Define preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Example usage: replace with your train/val/test folders
train_dir = "/content/data/keras_png_slices_data/keras_png_slices_train"
val_dir   = "/content/data/keras_png_slices_data/keras_png_slices_validate"
test_dir  = "/content/data/keras_png_slices_data/keras_png_slices_test"

trainset = MRIDataset(train_dir, transform=transform)
valset   = MRIDataset(val_dir, transform=transform)
testset  = MRIDataset(test_dir, transform=transform)

train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
val_loader   = DataLoader(valset, batch_size=64, shuffle=False)
test_loader  = DataLoader(testset, batch_size=64, shuffle=False)

print(f"Train: {len(trainset)}, Val: {len(valset)}, Test: {len(testset)}")